# 05 - Visualización y Dashboard
## Laboratorio 2: Detección de Cambios Urbanos - Chaitén

Este notebook documenta las visualizaciones y el dashboard interactivo.

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from pathlib import Path

PROCESSED_DIR = Path('../data/processed')
VECTOR_DIR = Path('../data/vector')

## 1. Dashboard Streamlit

El dashboard interactivo está en `app/app.py` e incluye:

- ✅ Mapa interactivo con capas de cambio (Folium)
- ✅ Gráficos dinámicos con selección de fechas (Plotly)
- ✅ Comparador visual antes/después
- ✅ Descarga de resultados (CSV)

**Para ejecutar:**
```bash
streamlit run app/app.py
```

## 2. Mapa Interactivo (Ejemplo Estático)

In [ ]:
# Crear mapa con Folium
zonas_path = VECTOR_DIR / 'zonas_analisis.geojson'

if zonas_path.exists():
    zonas = gpd.read_file(zonas_path)
    
    # Centro del mapa
    centro = [zonas.geometry.centroid.y.mean(), zonas.geometry.centroid.x.mean()]
    
    m = folium.Map(location=centro, zoom_start=10, tiles='CartoDB positron')
    
    # Agregar zonas
    folium.GeoJson(
        zonas,
        style_function=lambda x: {
            'fillColor': 'blue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.3
        }
    ).add_to(m)
    
    display(m)
else:
    print("No hay archivo de zonas")

## 3. Comparador Antes/Después

In [ ]:
evol_path = PROCESSED_DIR / 'evolucion_temporal.csv'

if evol_path.exists():
    evol = pd.read_csv(evol_path)
    evol['fecha'] = pd.to_datetime(evol['fecha'])
    
    t1 = evol.iloc[0]
    t2 = evol.iloc[-1]
    
    print("=" * 50)
    print(f"COMPARACIÓN: {t1['fecha'].strftime('%Y-%m-%d')} vs {t2['fecha'].strftime('%Y-%m-%d')}")
    print("=" * 50)
    print(f"\nNDVI Medio:     {t1['ndvi_mean']:.4f} → {t2['ndvi_mean']:.4f} (Δ {t2['ndvi_mean']-t1['ndvi_mean']:+.4f})")
    print(f"NDBI Medio:     {t1['ndbi_mean']:.4f} → {t2['ndbi_mean']:.4f} (Δ {t2['ndbi_mean']-t1['ndbi_mean']:+.4f})")
    print(f"% Vegetación:   {t1['pct_veg']:.1f}% → {t2['pct_veg']:.1f}% (Δ {t2['pct_veg']-t1['pct_veg']:+.1f}%)")
    print(f"% Urbano:       {t1['pct_urb']:.1f}% → {t2['pct_urb']:.1f}% (Δ {t2['pct_urb']-t1['pct_urb']:+.1f}%)")
else:
    print("No hay datos temporales")

## 4. Exportar para Video/Presentación

In [ ]:
# Crear figura resumen para captura
stats_path = PROCESSED_DIR / 'estadisticas_cambio.csv'

if stats_path.exists():
    stats = pd.read_csv(stats_path)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Gráfico de barras
    if 'urbanizacion_ha' in stats.columns:
        top5 = stats.nlargest(5, 'urbanizacion_ha')
        axes[0].bar(top5['zona'].astype(str), top5['urbanizacion_ha'], color='coral')
        axes[0].set_title('Top 5 Zonas - Urbanización (ha)')
        axes[0].tick_params(axis='x', rotation=45)
    
    # Pie chart de tipos de cambio
    if all(c in stats.columns for c in ['urbanizacion_ha', 'perdida_veg_ha', 'ganancia_veg_ha']):
        totales = [
            stats['urbanizacion_ha'].sum(),
            stats['perdida_veg_ha'].sum(),
            stats['ganancia_veg_ha'].sum()
        ]
        labels = ['Urbanización', 'Pérdida Veg.', 'Ganancia Veg.']
        colors = ['red', 'orange', 'green']
        axes[1].pie(totales, labels=labels, colors=colors, autopct='%1.1f%%')
        axes[1].set_title('Distribución de Cambios')
    
    plt.tight_layout()
    plt.savefig('../outputs/figures/resumen_cambios.png', dpi=150)
    print("Guardado: outputs/figures/resumen_cambios.png")
    plt.show()
else:
    print("No hay estadísticas")

## 5. Instrucciones del Dashboard

1. Ejecutar: `streamlit run app/app.py`
2. Abrir: http://localhost:8501
3. Usar sidebar para:
   - Seleccionar tipos de cambio
   - Elegir fechas de comparación
   - Descargar datos CSV
4. Explorar pestañas de gráficos y mapas

---
**Anterior**: [04_analisis_zonal.ipynb](04_analisis_zonal.ipynb)

---

## Flujo Completo

1. [01_descarga_datos.ipynb](01_descarga_datos.ipynb) - Adquisición
2. [02_calculo_indices.ipynb](02_calculo_indices.ipynb) - Índices espectrales
3. [03_deteccion_cambios.ipynb](03_deteccion_cambios.ipynb) - Detección
4. [04_analisis_zonal.ipynb](04_analisis_zonal.ipynb) - Cuantificación
5. **Este notebook** - Visualización